This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [ ]:
# Import the required packeages
import pandas as pd
import os
import json
import datetime
import concurrent.futures
from dotenv import load_dotenv
from pathlib import Path
from dateutil.relativedelta import relativedelta

### Import Scripts

In [ ]:
# Import model index functions
from pyprediktormapclient.model_index import ModelIndex

# Import OPC UA functions
from pyprediktormapclient.opc_ua import OPC_UA

# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

# import AUTH_CLIENT
from pyprediktormapclient.auth_client import AUTH_CLIENT

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path(".env")
load_dotenv(dotenv_path=dotenv_path)

username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]


In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

In [ ]:
# Connecting to ModelIndex APIs 
model = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

# Listed sites on the model index api server
namespaces = model.get_namespace_array()
# Types of Objects
object_types_json = model.get_object_types()
object_types = AnalyticsHelper(object_types_json)
namespace_list = object_types.namespaces_as_list(namespaces)

# Initate the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client)

### Download data from modelindex api

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get the objects of a type
sites_json = model.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the second site
first_site_id = sites.list_of_ids()[0]
# first_site_id = '14:1:BE.DK-ADU'
first_site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model.get_object_descendants(
    "StringSetType", [first_site_id], "PV_Assets"
)
string_sets = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets.dataframe

In [ ]:
# 1 day aggregated historical inverter data in asyncio process
one_days_historic_inverter_data2 = await opc_data.get_historical_aggregated_values_async(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=string_sets.variables_as_list(["DCPower"]),
    batch_size=500
)
one_days_historic_inverter_data2

### Stringset data

In [ ]:
def get_historical_aggregated_values(opc_data,
    start_time, 
    end_time, 
    pro_interval, 
    agg_name, 
    variable_list
) -> pd.DataFrame:
    
    vars = opc_data._get_variable_list_as_list(variable_list)
    extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in vars]

    body = {
        **opc_data.body, 
        "StartTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"), 
        "EndTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"), 
        "ProcessingInterval": pro_interval, 
        "AggregateName": agg_name,
        "ReadValueIds": extended_variables
    }
    print(body)

    content = request_from_api(
        rest_url=opcua_rest_url, 
        method="POST", 
        endpoint="values/historicalaggregated", 
        data=json.dumps(body, default=opc_data.json_serial), 
        headers=opc_data.headers, 
        extended_timeout=True
    )
    print(content)
    df_result = pd.json_normalize(
        content, 
        record_path=['HistoryReadResults', 'DataValues'], 
        meta=[['HistoryReadResults', 'NodeId', 'IdType'], ['HistoryReadResults', 'NodeId','Id'],['HistoryReadResults', 'NodeId','Namespace']
        ]
    )
    columns = {
        "Value.Type": "ValueType",
        "Value.Body": "Value",
        "StatusCode.Symbol": "StatusSymbol",
        "StatusCode.Code": "StatusCode",
        "SourceTimestamp": "Timestamp",
        "HistoryReadResults.NodeId.IdType": "IdType",
        "HistoryReadResults.NodeId.Id": "Id",
        "HistoryReadResults.NodeId.Namespace": "Namespace",
    }
    return opc_data._process_df(df_result, columns)

In [ ]:
start_time=(datetime.datetime.now() - datetime.timedelta(30))
end_time=(datetime.datetime.now() - datetime.timedelta(29))
pro_interval=600000
agg_name="Average"
variable_list=string_sets.variables_as_list(["DCPower"])

In [ ]:
def get_historical_aggregated_values(opc_data,
                                     start_time, 
                                     end_time, 
                                     pro_interval, 
                                     agg_name, 
                                     variable_list) -> pd.DataFrame:
    vars = opc_data._get_variable_list_as_list(variable_list)
    batch_size = 100
    batches = [vars[i:i + batch_size] for i in range(0, len(vars), batch_size)]
    
    combined_df = pd.DataFrame()  
    
    for batch in batches:
        extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in batch]
        
        body = {
            **opc_data.body, 
            "StartTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"), 
            "EndTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"), 
            "ProcessingInterval": pro_interval, 
            "AggregateName": agg_name,
            "ReadValueIds": extended_variables
        }
        
        content = request_from_api(
            rest_url=opcua_rest_url, 
            method="POST", 
            endpoint="values/historicalaggregated", 
            data=json.dumps(body, default=opc_data.json_serial), 
            headers=opc_data.headers, 
            extended_timeout=True
        )
        
        df_result = pd.json_normalize(
            content, 
            record_path=['HistoryReadResults', 'DataValues'], 
            meta=[['HistoryReadResults', 'NodeId', 'IdType'], ['HistoryReadResults', 'NodeId','Id'],['HistoryReadResults', 'NodeId','Namespace']]
        )
        
        if combined_df.empty:
            combined_df = df_result
        else:
            combined_df = pd.concat([combined_df, df_result], ignore_index=True)
    
    columns = {
        "Value.Type": "ValueType",
        "Value.Body": "Value",
        "StatusCode.Symbol": "StatusSymbol",
        "StatusCode.Code": "StatusCode",
        "SourceTimestamp": "Timestamp",
        "HistoryReadResults.NodeId.IdType": "IdType",
        "HistoryReadResults.NodeId.Id": "Id",
        "HistoryReadResults.NodeId.Namespace": "Namespace",
    }
    
    return opc_data._process_df(combined_df, columns)

In [ ]:
get_historical_aggregated_values(opc_data,
                                     start_time, 
                                     end_time, 
                                     pro_interval, 
                                     agg_name, 
                                     variable_list)

In [ ]:
import hashlib
import concurrent.futures

In [ ]:
def get_historical_aggregated_values(opc_data, start_time, end_time, pro_interval, agg_name, variable_list) -> pd.DataFrame:
    vars = opc_data._get_variable_list_as_list(variable_list)
    batch_size = 150
    batches = [vars[i:i + batch_size] for i in range(0, len(vars), batch_size)]

    def process_batch(batch):
        extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in batch]
        body = {
            **opc_data.body,
            "StartTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "EndTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "ProcessingInterval": pro_interval,
            "AggregateName": agg_name,
            "ReadValueIds": extended_variables
        }
        content = request_from_api(
            rest_url=opcua_rest_url,
            method="POST",
            endpoint="values/historicalaggregated",
            data=json.dumps(body, default=opc_data.json_serial),
            headers=opc_data.headers,
            extended_timeout=True
        )
        return pd.json_normalize(
            content,
            record_path=['HistoryReadResults', 'DataValues'],
            meta=[['HistoryReadResults', 'NodeId', 'IdType'], ['HistoryReadResults', 'NodeId', 'Id'], ['HistoryReadResults', 'NodeId', 'Namespace']]
        )

    dataframes = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_batch = {executor.submit(process_batch, batch): batch for batch in batches}
        for future in concurrent.futures.as_completed(future_to_batch):
            dataframes.append(future.result())

    combined_df = pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

    columns = {
        "Value.Type": "ValueType",
        "Value.Body": "Value",
        "StatusCode.Symbol": "StatusSymbol",
        "StatusCode.Code": "StatusCode",
        "SourceTimestamp": "Timestamp",
        "HistoryReadResults.NodeId.IdType": "IdType",
        "HistoryReadResults.NodeId.Id": "Id",
        "HistoryReadResults.NodeId.Namespace": "Namespace",
    }

    return opc_data._process_df(combined_df, columns)

In [ ]:
vars = opc_data._get_variable_list_as_list(variable_list)
extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in vars]

In [ ]:
body = {
    **opc_data.body,
    "StartTime": start_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
    "EndTime": end_time.strftime("%Y-%m-%dT%H:%M:%SZ"),
    "ProcessingInterval": pro_interval,
    "AggregateName": agg_name,
    "ReadValueIds": extended_variables
}
body

In [ ]:
get_historical_aggregated_values(opc_data,
                                     start_time, 
                                     end_time, 
                                     pro_interval, 
                                     agg_name, 
                                     variable_list)

In [ ]:
start_time = datetime.now() - relativedelta(months=1)
end_time = datetime.now()
get_historical_aggregated_values(opc_data,
                                     start_time, 
                                     end_time, 
                                     pro_interval, 
                                     agg_name, 
                                     variable_list)

In [ ]:
# History data for 1 day, 10 min aggregate - stringsets
history_agg = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=600000,
    agg_name="Average",
    variable_list=inverters.variables_as_list(["DCPower"]),
)
history_agg

In [ ]:
import copy
import math
from pydantic import BaseModel, AnyUrl
from datetime import timedelta
import asyncio
import aiohttp

In [ ]:
class Variables(BaseModel):
    """Helper class to parse all values api's.
    Variables are described in https://reference.opcfoundation.org/v104/Core/docs/Part3/8.2.1/

        Variables:
            Id: str - Id of the signal, e.g. SSO.EG-AS.WeatherSymbol
            Namespace: int - Namespace on the signal, e.g. 2.
            IdType: int - IdTypes described in https://reference.opcfoundation.org/v104/Core/docs/Part3/8.2.3/.
    """
    Id: str
    Namespace: int
    IdType: int

In [ ]:
async def make_async_api_request(opc_data, start_time: datetime, end_time: datetime, pro_interval: int, agg_name: str, variable_list: list[Variables]) -> dict:
        """Make API request for the given time range and variable list"""

        # Creating a new variable list to remove pydantic models
        vars = opc_data._get_variable_list_as_list(variable_list)

        extended_variables = [
            {
                    "NodeId": var,
                    "AggregateName": agg_name,
            }
            for var in vars
        ]
        body = copy.deepcopy(opc_data.body)
        body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
        body["ProcessingInterval"] = pro_interval
        body["ReadValueIds"] = extended_variables
        body["AggregateName"] = agg_name

            # Make API request using aiohttp session
        async with aiohttp.ClientSession() as session:
            async with session.post(
                f"{opcua_rest_url}values/historicalaggregated",
                data=json.dumps(body, default=opc_data.json_serial),
                headers=opc_data.headers,
                timeout=aiohttp.ClientTimeout(total=None)  
            ) as response:
                response.raise_for_status()
                content = await response.json()

        return content

In [ ]:
vars = opc_data._get_variable_list_as_list(variable_list)
vars1 = vars[0:5]

In [ ]:
extended_variables = [
            {
                    "NodeId": var,
                    "AggregateName": agg_name,
            }
            for var in vars1
]
len(extended_variables)

In [ ]:
body = copy.deepcopy(opc_data.body)
body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
body["ProcessingInterval"] = pro_interval
body["ReadValueIds"] = extended_variables
body["AggregateName"] = agg_name
body

In [ ]:
f"{opcua_rest_url}values/historicalaggregated",

In [ ]:
data=json.dumps(body, default=opc_data.json_serial)

In [ ]:
data

In [ ]:
data_dict = json.loads(data)

In [ ]:
read_value_ids = data_dict['ReadValueIds']

In [ ]:
len(read_value_ids)

In [ ]:
headers=opc_data.headers
headers

In [ ]:
timeout=aiohttp.ClientTimeout(total=None) 
timeout

In [ ]:
async with aiohttp.ClientSession() as session:
    async with session.post(
        f"{opcua_rest_url}values/historicalaggregated",
        data=json.dumps(body, default=opc_data.json_serial),
        headers=opc_data.headers,
        timeout=aiohttp.ClientTimeout(total=None)  
    ) as response:
        response.raise_for_status()
        content = await response.json()

In [ ]:
content

In [ ]:
def generate_time_batches(start_time: datetime, end_time: datetime, pro_interval: int, batch_size: int) -> list[tuple]:
    """Generate time batches based on start time, end time, processing interval, and batch size"""

    total_time_range = end_time - start_time
    pro_interval_seconds = (pro_interval / 1000)
    total_data_points = (total_time_range.total_seconds() // pro_interval_seconds) + 1

    total_batches = math.ceil(total_data_points / batch_size)
    actual_batch_size = math.ceil(total_data_points / total_batches)

    time_batches = [
        (start_time + timedelta(seconds=(i * actual_batch_size * pro_interval_seconds)),
        start_time + timedelta(seconds=((i + 1) * actual_batch_size * pro_interval_seconds)) - timedelta(seconds=pro_interval_seconds))
        for i in range(total_batches)
    ]

    return time_batches

In [ ]:
def generate_variable_batches(variable_list: list[Variables], batch_size: int) -> list[list[Variables]]:
    """Generate variable batches based on the variable list and batch size"""

    variable_batches = [
        variable_list[i:i + batch_size] for i in range(0, len(variable_list), batch_size)
    ]

    return variable_batches

In [ ]:
def process_api_response(opc_data, response: dict) -> pd.DataFrame:
        """Process the API response and return the result dataframe"""
        
        df_result = pd.json_normalize(response, record_path=['HistoryReadResults', 'DataValues'], 
                                      meta=[['HistoryReadResults', 'NodeId', 'IdType'], ['HistoryReadResults', 'NodeId','Id'],
                                            ['HistoryReadResults', 'NodeId','Namespace']] )

        for i, row in df_result.iterrows():
            if not math.isnan(row["Value.Type"]):
                value_type = opc_data._get_value_type(int(row["Value.Type"])).get("type")
                df_result.at[i, "Value.Type"] = str(value_type)

        df_result.rename(
            columns={
                "Value.Type": "ValueType",
                "Value.Body": "Value",
                "StatusCode.Symbol": "StatusSymbol",
                "StatusCode.Code": "StatusCode",
                "SourceTimestamp": "Timestamp",
                "HistoryReadResults.NodeId.IdType": "Id",
                "HistoryReadResults.NodeId.Namespace": "Namespace",
            },
            errors="raise",
            inplace=True,
        )

        return df_result

In [ ]:
async def get_historical_aggregated_values_async(
    opc_data,
    start_time: datetime,
    end_time: datetime,
    pro_interval: int,
    agg_name: str,
    variable_list: list[Variables],
    batch_size: int = 1000
) -> pd.DataFrame:
    """Request historical aggregated values from the OPC UA server with batching"""

    
    time_batches = generate_time_batches(start_time, end_time, pro_interval, batch_size)
    variable_batches = generate_variable_batches(variable_list, batch_size)

    # Creating tasks for each API request and gathering the results
    tasks = []

    for time_batch_start, time_batch_end in time_batches:
        for variable_sublist in variable_batches:
            task = asyncio.create_task(
                make_async_api_request(opc_data, time_batch_start, time_batch_end, pro_interval, agg_name, variable_sublist)
            ) 
            tasks.append(task)
    
    # Execute all tasks concurrently and gather their results
    responses = await asyncio.gather(*tasks)
    
    # Processing the API responses
    result_list = []
    for idx, batch_response in enumerate(responses):
        
        batch_result = process_api_response(opc_data, batch_response)
        result_list.append(batch_result)
       
    result_df = pd.concat(result_list, ignore_index=True)

    return result_df

In [ ]:
# 1 day aggregated historical inverter data in asyncio process
one_days_historic_inverter_data2 = await get_historical_aggregated_values_async(
    opc_data,
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=string_sets.variables_as_list(["DCPower"]),
    batch_size=100
)
one_days_historic_inverter_data2

In [ ]:
async def generate_time_chunks(start_time: datetime, end_time: datetime):
    """Generate time chunks between start_time and end_time, each chunk_duration_minutes long."""
    delta = timedelta(minutes=60)
    current_time = start_time
    while current_time < end_time:
        chunk_end_time = min(current_time + delta, end_time)
        yield (current_time, chunk_end_time)
        current_time = chunk_end_time

In [ ]:
async def make_async_api_request(opc_data, start_time: datetime, end_time: datetime, pro_interval: int, agg_name: str, variable_list: list[Variables], max_data_points=500) -> dict:
    """Make API request for the given time range and variable list, with additional chunking based on data points."""

    def chunk_list(lst, n):
        """Yield successive n-sized chunks from lst."""
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    async def fetch_data_for_time_period(session, vars_chunk, start, end):
        """Fetch data for a given time period and chunk of variables."""
        extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in vars_chunk]
        body = copy.deepcopy(opc_data.body)
        body["StartTime"] = start.strftime("%Y-%m-%dT%H:%M:%SZ")
        body["EndTime"] = end.strftime("%Y-%m-%dT%H:%M:%SZ")
        body["ProcessingInterval"] = pro_interval
        body["ReadValueIds"] = extended_variables
        body["AggregateName"] = agg_name

        async with session.post(
            f"{opcua_rest_url}values/historicalaggregated",
            data=json.dumps(body, default=str),
            headers=opc_data.headers,
            timeout=aiohttp.ClientTimeout(total=None)
        ) as response:
            response.raise_for_status()
            return await response.json()

    # Creating a new variable list to remove pydantic models
    vars = opc_data._get_variable_list_as_list(variable_list)
    chunk_size = 5  # Chunk size for node IDs
    vars_chunks = list(chunk_list(vars, chunk_size))

    all_responses = []
    async with aiohttp.ClientSession() as session:
        for vars_chunk in vars_chunks:
            # Generate time chunks for the given time period
            async for start, end in generate_time_chunks(start_time, end_time):
                content = await fetch_data_for_time_period(session, vars_chunk, start, end)
                all_responses.append(content)
    return all_responses

In [ ]:
async def make_async_api_request(opc_data, start_time: datetime, end_time: datetime, pro_interval: int, agg_name: str, variable_list: list[Variables]) -> dict:
    """Make API request for the given time range and variable list"""

    def chunk_list(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # Creating a new variable list to remove pydantic models
    vars = opc_data._get_variable_list_as_list(variable_list)

    chunk_size = 150  
    vars_chunks = list(chunk_list(vars, chunk_size))

    all_responses = []
    async with aiohttp.ClientSession() as session:
        for vars_chunk in vars_chunks:
            extended_variables = [{"NodeId": var, "AggregateName": agg_name} for var in vars_chunk]
            body = copy.deepcopy(opc_data.body)
            body["StartTime"] = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
            body["EndTime"] = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
            body["ProcessingInterval"] = pro_interval
            body["ReadValueIds"] = extended_variables
            body["AggregateName"] = agg_name

            async with session.post(
                f"{opcua_rest_url}values/historicalaggregated",
                data=json.dumps(body, default=str),
                headers=opc_data.headers,
                timeout=aiohttp.ClientTimeout(total=None)
            ) as response:
                response.raise_for_status()
                content = await response.json()
                all_responses.append(content)  

    return all_responses

In [ ]:
from datetime import datetime, timedelta
from typing import List, Tuple

In [ ]:
def generate_time_chunks(start_time: datetime, end_time: datetime, interval_hours: int) -> List[Tuple[datetime, datetime]]:
    """Generate time chunks within the given start and end time with specified interval in hours."""
    delta = timedelta(hours=interval_hours)
    current_time = start_time
    chunks = []

    while current_time < end_time:
        chunk_end_time = min(current_time + delta, end_time)  
        chunks.append((current_time, chunk_end_time))
        current_time += delta

    return chunks

In [ ]:
# 1 day aggregated historical inverter data in asyncio process
one_days_historic_inverter_data2 = await make_async_api_request(
    opc_data,
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=60*1000,
    agg_name="Average",
    variable_list=string_sets.variables_as_list(["DCPower"])
)
one_days_historic_inverter_data2